In [13]:
import pandas as pd
import numpy as np
import sumu
import pandas as pd
import numpy as np
from glmnet import ElasticNet
import random
import time
import random

In [26]:
def generate_dataset_and_dag(dim,number, deg):
    dictionary = {}
    for x in range(1,dim+1):
        dictionary[x] = np.random.normal(0,1,number)
    df = pd.DataFrame(dictionary)
    possible_children =set(range(1,dim+1))
    possible_parents =set()
    root = random.sample(possible_children, k=1)[0]
    possible_children.remove(root)
    possible_parents.add(root)
    dag = dict()
    for x in range(1,dim+1):
        dag[x]=[]
    for x in range(1,dim):
        if len(possible_children) !=0:
            new_children = list(set(random.choices(list(possible_children), k=deg)))
        else:
            break
        their_parent = random.sample(possible_parents, k=1)[0]
        relations = list()
        for child in new_children:
            possible_parents.add(child)
            possible_children.remove(child)
            rand_weight = random.random()
            df[child] = df[their_parent]*rand_weight
            relations.append((child,rand_weight))
        dag[their_parent]+=relations
    
    return np.array(df),dag
        
        
arr1, dag1 = generate_dataset_and_dag(5,100000,2)        

In [27]:
print(dag1)

{1: [], 2: [(1, 0.9021331914901047)], 3: [(4, 0.6575118506823157)], 4: [], 5: [(2, 0.4124263913430325), (3, 0.2928343189593734)]}


In [ ]:
start = time.time()
data = sumu.Data(arr1, discrete=False)
params = {

          "array": arr1,
          "data": data,
          "scoref": "bge",  # Or "bdeu" for discrete data.
          # "ess": 10,        # If using BDeu.
          "max_id": -1,
          "K": 2,
          "d": 2,
          "cp_algo": "greedy-lite",
          "mc3_chains": 10,
          "burn_in": 100,
          "iterations": 10000,
          "thinning": 10}

g = sumu.Gadget(**params)
dag = g.sample()
end = time.time()
print(dag)
print(dag1)
print(end - start)


In [ ]:
for x in dag:
    if x in dag1:
        print('______',x)
        print(dag1[x])
        print(dag[x])
    